In [2]:
import torch
from ultralytics import YOLO

In [3]:
# ✅ Check GPU
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
# ✅ Load YOLOv8 model (small version for faster training)
model = YOLO("yolov8n.pt")   # you can change to yolov8s.pt or yolov8m.pt for higher accuracy

# ✅ Train the model
results = model.train(
    data=r"D:\VS code\Final Project\Guvi_Project.v2i.yolov8\data.yaml",  # your dataset YAML
    epochs=50,          # increase to 100+ for better results
    batch=16,           # adjust if you hit OOM (Out of Memory)
    imgsz=640,          # resize images
    device=0,  # use GPU if available
    workers=0,          # Windows fix to avoid multiprocessing crash
    project="runs/train",
    name="helmet_accident_yolov8"
)

# ✅ Save the best model
print("✅ Training complete! Best model saved at:", results.save_dir)


Ultralytics 8.3.202  Python-3.12.6 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\VS code\Final Project\Guvi_Project.v2i.yolov8\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=helmet_accident_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

In [6]:
from ultralytics import YOLO
import torch
import os

# ✅ Path to trained model
MODEL_PATH = r"D:\MDTE21\FINAL PROJECT\runs\train\helmet_accident_yolov8\weights\best.pt"

# ✅ Path to test image/video (or use folder)
SOURCE = r"D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images"
# SOURCE = 0   # <-- Uncomment this for live webcam

# ✅ Output folder
OUTPUT_DIR = r"D:\MDTE21\FINAL PROJECT\runs\inference_results"

# ✅ Load model
device = 0 if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = YOLO(MODEL_PATH)

# ✅ Run prediction
results = model.predict(
    source=SOURCE,
    save=True,             # save annotated results
    project=OUTPUT_DIR,    # save directory
    name="test_run",       # run name
    device=device,
    conf=0.5               # confidence threshold
)

print(f"✅ Inference complete! Results saved in {OUTPUT_DIR}\\test_run")

Using device: 0

image 1/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\16_flip70_jpg.rf.96d4bbba98e51c5c3cc7adae3bfd976c.jpg: 640x640 1 Accident, 60.5ms
image 2/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\18_flip70_jpg.rf.7b848e83785d4b13c229f6deafec1eec.jpg: 640x640 1 Accident, 1 Helmet, 77.5ms
image 3/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\20_flip70_jpg.rf.02309e7e1ad5c0a582beaefa83111fdc.jpg: 640x640 2 Accidents, 7.4ms
image 4/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\240_F_250520479_FdSJwJtwRiiCzVIIu699vOsZCkSqP4VV_jpg.rf.1299a307cdf2ca6808d1864e7a888359.jpg: 640x640 (no detections), 8.3ms
image 5/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\240_F_618729965_jbcjMgjADfBHgkPXQPg2ii9yTzeGVyPa_flip30_jpg.rf.a2d158e5ced7c1036e4ba0005d2c0662.jpg: 640x640 (no detections), 11.8ms
image 6/228 D:\MDTE21\FINAL PROJECT\Guvi_Project.v2i.yolov8\valid\images\240_F_619320642_RYXJopjzd0EUgII

In [11]:
from ultralytics import YOLO
import cv2, json, time, os

# ✅ Load your trained model
MODEL_PATH = r"D:\MDTE21\FINAL PROJECT\runs\train\helmet_accident_yolov8\weights\best.pt"
model = YOLO(MODEL_PATH)

# ✅ Results folder
os.makedirs("results", exist_ok=True)

def process_frame(frame, camera="cam1"):
    results = model(frame)   # Run YOLO
    r = results[0]
    detections = []
    annotated = r.plot()

    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        xyxy = box.xyxy[0].tolist()
        detections.append({
            "class": model.names[cls],   # human-readable class name
            "conf": conf,
            "bbox": xyxy
        })

    ts = int(time.time())

    # Save detections JSON
    with open(f"results/detections_{ts}.json", "w") as f:
        json.dump(detections, f)

    return annotated, detections, ts

# ✅ Load your test video (change path here if needed)
VIDEO_PATH = r"D:\MDTE21\FINAL PROJECT\input.mp4"  # <-- replace with your input video
cap = cv2.VideoCapture(VIDEO_PATH)

out = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    ann, dets, ts = process_frame(frame)

    if out is None:
        out = cv2.VideoWriter("results/output.mp4",
                              cv2.VideoWriter_fourcc(*"mp4v"),
                              int(cap.get(cv2.CAP_PROP_FPS)),
                              (frame.shape[1], frame.shape[0]))
    out.write(ann)

    cv2.imshow("SafeRide AI - Results", ann)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
